<a href="https://colab.research.google.com/github/Anechka2212/task-for-avito/blob/master/Avito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт необходимых для работы библиотек

In [0]:
import requests
import bs4
import scipy as sc
import numpy as np
import pandas as pd
import nltk
import glob

from google.colab import drive

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from typing import Dict, Tuple
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from scipy import sparse

Монтирование гугл диска 

In [2]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


Загрузка данных с гугл диска

In [0]:
train_data = pd.read_csv('/content/gdrive/My Drive/train.csv', sep=",")

In [0]:
train_data.head(10)

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1
5,5,Бас гитара invasion bg110,Состояние хорошее. Имеется теплый чехол .,3999.0,50
6,6,"Смесь ""Грудничок"" г. Зеленодольск",Смесь молочная адаптированная ультрапастеризов...,15.0,41
7,7,G-shock,Часы абсолютно новые! с коробкой. Часы Китай...,2500.0,36
8,8,"Санатории Белоруссии. - ""Лепельский военный""",Санатории Белоруссии! - «Лепельский военный» ...,1090.0,48
9,9,Фотохолст,Фотохолст на подрамнике. 36х58см. Галерейная н...,1250.0,19


In [0]:
train_data.shape

(489517, 5)

In [0]:
train_data.isnull().sum()

item_id        0
title          0
description    0
price          0
category_id    0
dtype: int64

In [4]:
test_data = pd.read_csv('/content/gdrive/My Drive/test.csv', sep=",")
test_data.head(10)

,item_id,title,description,price
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
3,489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
4,489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0
5,489522,Дверь Profil Doors Х21,Дверь межкомнатная ProfilDoors 21Х. Серия Х пр...,6600.0
6,489523,"Новый Cort CR100 в отличном состоянии, с коробкой","Доброго времени суток, Томичи. Срочно нужны де...",13900.0
7,489524,Кресла,Продаются 2кресла в хорошем состоянии за два 600,600.0
8,489525,Сверх быстрый YotaPhone 2,"Да , разбит дисплей , но когда экран работает ...",6000.0
9,489526,Люстра ArteLamp medici A9468LM-5BR,ВНИМАНИЕ РАСПРОДАЖА !!!! Люстра ArteLamp MEDIC...,3650.0


In [0]:
test_data.shape

(243166, 4)

In [0]:
test_data.isnull().sum()

item_id        0
title          0
description    0
price          0
dtype: int64

In [5]:
categories = pd.read_csv('/content/gdrive/My Drive/category.csv', sep=",")
categories.head(10)

,category_id,name
0,0,Бытовая электроника|Телефоны|iPhone
1,1,Бытовая электроника|Ноутбуки
2,2,Бытовая электроника|Телефоны|Samsung
3,3,Бытовая электроника|Планшеты и электронные кни...
4,4,"Бытовая электроника|Игры, приставки и программ..."
5,5,Бытовая электроника|Аудио и видео|Телевизоры и...
6,6,Бытовая электроника|Телефоны|Другие марки
7,7,Бытовая электроника|Настольные компьютеры
8,8,"Бытовая электроника|Игры, приставки и программ..."
9,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...


In [0]:
categories.shape

(54, 2)

In [0]:
categories.isnull().sum()

category_id    0
name           0
dtype: int64

Создание столбца для будущего различия train и test данных

In [0]:
isTrain = []
for i in train_data['item_id']:
  isTrain.append(1)
boolTrain = pd.DataFrame({'isTrain': isTrain})

isNotTrain = []
for i in test_data['item_id']:
  isNotTrain.append(0)
notBoolTrain = pd.DataFrame({'isTrain': isNotTrain})


In [0]:
train_data.loc[:,'isTrain'] = boolTrain
train_data.head(5)

,item_id,title,description,price,category_id,isTrain
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,1
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,1
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,1
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,1
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,1


In [0]:
test_data.loc[:,'isTrain'] = notBoolTrain
test_data.head(5)

,item_id,title,description,price,isTrain
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0,0
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0,0
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0,0
3,489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0,0
4,489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0,0


Формирование словарей для обучения моделей

In [6]:
vocab_title_train_list = train_data['title'].tolist()
vocab_title_test_list = test_data['title'].tolist()
vocab_title_train_list[0:5]

['Картина',
 'Стулья из прессованной кожи',
 'Домашняя мини баня',
 'Эксклюзивная коллекция книг "Трансаэро" + подарок',
 'Ноутбук aser']

In [7]:
concat_cols = ['title', 'description', 'price']
frames = [train_data[concat_cols], test_data[concat_cols]]
df_concat_text = pd.concat(frames)
df_concat_text.head()

,title,description,price
0,Картина,Гобелен. Размеры 139х84см.,1000.0
1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0
2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0
3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0
4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0


In [0]:
assert train_data.shape[0] + test_data.shape[0] == df_concat_text.shape[0]

In [0]:
df_concat_text.shape

(732683, 3)

Векторизация текстов

In [0]:
def vectorize_text(col):
  v = TfidfVectorizer()
  features = v.fit_transform(df_concat_text[col])
  return features

In [10]:
vect_title = vectorize_text('title')
vect_title

<732683x196641 sparse matrix of type '<class 'numpy.float64'>'
	with 2684046 stored elements in Compressed Sparse Row format>

In [11]:
vect_description = vectorize_text('description')
vect_description

<732683x579563 sparse matrix of type '<class 'numpy.float64'>'
	with 22248631 stored elements in Compressed Sparse Row format>

Текстовые признаки нормированы. Поэтому следует также отнормировать стоимость

In [12]:
np.min(vect_title), np.max(vect_title), np.min(vect_description), np.max(vect_description)

(0.0, 1.0, 0.0, 1.0)

In [0]:
price_scaler = MinMaxScaler()
price_scaled_data = price_scaler.fit_transform(df_concat_text['price'].values.reshape(-1, 1))

In [0]:
price_scaled_data.shape

(732683, 1)

In [0]:
np.min(price_scaled_data), np.max(price_scaled_data)

(0.0, 1.0)

Объединим данные обучающей и тестовой выборок

In [14]:
train_array = sparse.hstack([vect_title[0:train_data.shape[0], :], vect_description[0:train_data.shape[0], :], sparse.csr_matrix(price_scaled_data[0:train_data.shape[0], :])])
train_array

<489517x776205 sparse matrix of type '<class 'numpy.float64'>'
	with 17160166 stored elements in COOrdinate format>

In [15]:
test_array = sparse.hstack([vect_title[train_data.shape[0]:, :], vect_description[train_data.shape[0]:, :], sparse.csr_matrix(price_scaled_data[train_data.shape[0]:, :])])
test_array

<243166x776205 sparse matrix of type '<class 'numpy.float64'>'
	with 8505116 stored elements in COOrdinate format>

Создадим простейшую модель без учета иерархии категорий. Поскольку тестовая выборка не содержит категорий для проверки, то проверим качество модели с использованием кросс-валидации.

In [16]:
train_y=train_data['category_id'].values
train_y

array([19, 22, 37, ..., 25,  9, 50])

In [17]:
lr = LogisticRegression()
scores = cross_val_score(lr, train_array, y=train_y, scoring='accuracy', cv=3)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [18]:
np.mean(scores)

0.8846271273817149

Исследование категорий

In [19]:
for c in categories['name'].values:
  print('{} - {}'.format(c, c.count("|")))

Бытовая электроника|Телефоны|iPhone - 2
Бытовая электроника|Ноутбуки - 1
Бытовая электроника|Телефоны|Samsung - 2
Бытовая электроника|Планшеты и электронные книги|Планшеты - 2
Бытовая электроника|Игры, приставки и программы|Игровые приставки - 2
Бытовая электроника|Аудио и видео|Телевизоры и проекторы - 2
Бытовая электроника|Телефоны|Другие марки - 2
Бытовая электроника|Настольные компьютеры - 1
Бытовая электроника|Игры, приставки и программы|Игры для приставок - 2
Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки - 3
Бытовая электроника|Телефоны|Nokia - 2
Бытовая электроника|Телефоны|Sony - 2
Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы - 2
Бытовая электроника|Товары для компьютера|Мониторы - 2
Бытовая электроника|Товары для компьютера|Сетевое оборудование - 2
Для дома и дачи|Ремонт и строительство|Стройматериалы - 2
Для дома и дачи|Мебель и интерьер|Кровати, диваны и кресла - 2
Для дома и дачи|Ремонт и строительство|Инструменты - 2
Для дома и дачи|Мебель и инте

In [0]:
categories

,category_id,name
0,0,Бытовая электроника|Телефоны|iPhone
1,1,Бытовая электроника|Ноутбуки
2,2,Бытовая электроника|Телефоны|Samsung
3,3,Бытовая электроника|Планшеты и электронные кни...
4,4,"Бытовая электроника|Игры, приставки и программ..."
5,5,Бытовая электроника|Аудио и видео|Телевизоры и...
6,6,Бытовая электроника|Телефоны|Другие марки
7,7,Бытовая электроника|Настольные компьютеры
8,8,"Бытовая электроника|Игры, приставки и программ..."
9,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...


In [0]:
# определение уровня иерархии
def count_category_level(name):
  return name.count("|") + 1

In [0]:
categories['level'] = categories.apply(lambda x: count_category_level(x['name']), axis=1)

In [22]:
for index, c in categories.iterrows():
  print('{} \t {} \t {}'.format(c['level'], c['name'].count("|"), c['name']))

3 	 2 	 Бытовая электроника|Телефоны|iPhone
2 	 1 	 Бытовая электроника|Ноутбуки
3 	 2 	 Бытовая электроника|Телефоны|Samsung
3 	 2 	 Бытовая электроника|Планшеты и электронные книги|Планшеты
3 	 2 	 Бытовая электроника|Игры, приставки и программы|Игровые приставки
3 	 2 	 Бытовая электроника|Аудио и видео|Телевизоры и проекторы
3 	 2 	 Бытовая электроника|Телефоны|Другие марки
2 	 1 	 Бытовая электроника|Настольные компьютеры
3 	 2 	 Бытовая электроника|Игры, приставки и программы|Игры для приставок
4 	 3 	 Бытовая электроника|Телефоны|Аксессуары|Чехлы и плёнки
3 	 2 	 Бытовая электроника|Телефоны|Nokia
3 	 2 	 Бытовая электроника|Телефоны|Sony
3 	 2 	 Бытовая электроника|Аудио и видео|Акустика, колонки, сабвуферы
3 	 2 	 Бытовая электроника|Товары для компьютера|Мониторы
3 	 2 	 Бытовая электроника|Товары для компьютера|Сетевое оборудование
3 	 2 	 Для дома и дачи|Ремонт и строительство|Стройматериалы
3 	 2 	 Для дома и дачи|Мебель и интерьер|Кровати, диваны и кресла
3 	 2 	 Для дома

In [0]:
# Словарь для определения уровня на основе id категории
category_level_dict = {}
for index, c in categories.iterrows():
  id = c['category_id']
  level = c['level']
  category_level_dict[id] = level

In [62]:
category_level_dict

{0: 3,
 1: 2,
 2: 3,
 3: 3,
 4: 3,
 5: 3,
 6: 3,
 7: 2,
 8: 3,
 9: 4,
 10: 3,
 11: 3,
 12: 3,
 13: 3,
 14: 3,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 3,
 24: 2,
 25: 3,
 26: 4,
 27: 3,
 28: 3,
 29: 4,
 30: 3,
 31: 3,
 32: 3,
 33: 4,
 34: 3,
 35: 4,
 36: 3,
 37: 3,
 38: 4,
 39: 4,
 40: 3,
 41: 3,
 42: 3,
 43: 3,
 44: 3,
 45: 3,
 46: 3,
 47: 3,
 48: 3,
 49: 2,
 50: 3,
 51: 3,
 52: 3,
 53: 3}

In [0]:
train_data['level'] = train_data.apply(lambda x: category_level_dict[x['category_id']], axis=1)

In [0]:
train_data.head()

Построим отдельные модели для всех уровней иерархии

In [0]:
td2 = train_data[train_data['level'] == 2]
td3 = train_data[train_data['level'] == 3]
td4 = train_data[train_data['level'] == 4]

In [67]:
td2.shape

(35584, 6)

In [68]:
td3.shape

(390839, 6)

In [69]:
td4.shape

(63094, 6)

In [70]:
td2['category_id'].unique()

array([ 1, 24, 49,  7])

In [71]:
# номера строк отфильтрованных записей
td2.index

Int64Index([     4,     15,     34,     35,     40,     71,    119,    121,
               126,    130,
            ...
            489399, 489402, 489404, 489420, 489421, 489453, 489468, 489471,
            489481, 489483],
           dtype='int64', length=35584)

Дальше можно было бы отфильтровать разреженную матрицу по номерам строк, но, к сожалению, для разреженной матрицы не реализована такая операция. Единственный вариант - фильтровать исходный набор данных и каждый раз заново строить фичи.

Тестовая выборка не рассматривается, так как качество проверяем с помощью кросс-валидации для обучающей.

In [0]:
def vectorize_text_2(df, col):
  v = TfidfVectorizer()
  features = v.fit_transform(df[col])
  return features

In [72]:
td2_vect_title = vectorize_text_2(td2, 'title')
td2_vect_title

<35584x23225 sparse matrix of type '<class 'numpy.float64'>'
	with 135154 stored elements in Compressed Sparse Row format>

In [73]:
td3_vect_title = vectorize_text_2(td3, 'title')
td3_vect_title

<390839x133174 sparse matrix of type '<class 'numpy.float64'>'
	with 1449664 stored elements in Compressed Sparse Row format>

In [74]:
td4_vect_title = vectorize_text_2(td4, 'title')
td4_vect_title

<63094x20401 sparse matrix of type '<class 'numpy.float64'>'
	with 207139 stored elements in Compressed Sparse Row format>

In [75]:
td2_vect_description = vectorize_text_2(td2, 'description')
td2_vect_description

<35584x104762 sparse matrix of type '<class 'numpy.float64'>'
	with 1456780 stored elements in Compressed Sparse Row format>

In [76]:
td3_vect_description = vectorize_text_2(td3, 'description')
td3_vect_description

<390839x410768 sparse matrix of type '<class 'numpy.float64'>'
	with 12046508 stored elements in Compressed Sparse Row format>

In [77]:
td4_vect_description = vectorize_text_2(td4, 'description')
td4_vect_description

<63094x71062 sparse matrix of type '<class 'numpy.float64'>'
	with 1375453 stored elements in Compressed Sparse Row format>

In [78]:
td2_price_scaler = MinMaxScaler()
td2_price_scaled_data = td2_price_scaler.fit_transform(td2['price'].values.reshape(-1, 1))
td2_price_scaled_data.shape

(35584, 1)

In [79]:
td3_price_scaler = MinMaxScaler()
td3_price_scaled_data = td3_price_scaler.fit_transform(td3['price'].values.reshape(-1, 1))
td3_price_scaled_data.shape

(390839, 1)

In [80]:
td4_price_scaler = MinMaxScaler()
td4_price_scaled_data = td4_price_scaler.fit_transform(td4['price'].values.reshape(-1, 1))
td4_price_scaled_data.shape

(63094, 1)

In [81]:
td2_train_array = sparse.hstack([td2_vect_title, td2_vect_description, sparse.csr_matrix(td2_price_scaled_data)])
td2_train_array

<35584x127988 sparse matrix of type '<class 'numpy.float64'>'
	with 1627507 stored elements in COOrdinate format>

In [82]:
td3_train_array = sparse.hstack([td3_vect_title, td3_vect_description, sparse.csr_matrix(td3_price_scaled_data)])
td3_train_array

<390839x543943 sparse matrix of type '<class 'numpy.float64'>'
	with 13886974 stored elements in COOrdinate format>

In [83]:
td4_train_array = sparse.hstack([td4_vect_title, td4_vect_description, sparse.csr_matrix(td4_price_scaled_data)])
td4_train_array

<63094x91464 sparse matrix of type '<class 'numpy.float64'>'
	with 1645685 stored elements in COOrdinate format>

In [84]:
td2_train_y=td2['category_id'].values
td2_train_y.shape

(35584,)

In [85]:
td3_train_y=td3['category_id'].values
td3_train_y.shape

(390839,)

In [86]:
td4_train_y=td4['category_id'].values
td4_train_y.shape

(63094,)

In [88]:
scores = cross_val_score(LogisticRegression(), td2_train_array, y=td2_train_y, scoring='accuracy', cv=3)
print('LogisticRegression for 2 level')
print('Accuracy={}'.format(np.mean(scores)))
print('-------------')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression for 2 level
Accuracy=0.9832228108729119
-------------


In [89]:
scores = cross_val_score(LogisticRegression(), td3_train_array, y=td3_train_y, scoring='accuracy', cv=3)
print('LogisticRegression for 3 level')
print('Accuracy={}'.format(np.mean(scores)))
print('-------------')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression for 3 level
Accuracy=0.8849347813667187
-------------


In [90]:
scores = cross_val_score(LogisticRegression(), td4_train_array, y=td4_train_y, scoring='accuracy', cv=3)
print('LogisticRegression for 4 level')
print('Accuracy={}'.format(np.mean(scores)))
print('-------------')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression for 4 level
Accuracy=0.9628649236284911
-------------
